# Text Generation
## Character-Level LSTM in PyTorch

In this assignment, you will train a Reccurent Neural Network to generate a text one character at the time. The task is divided into steps for simpler navigation.

Let's start!

In [1]:
! nvidia-smi

Sun Nov 14 18:04:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Step 0. Data

As training data, we will use "War and Peace" by Leo Tolstoy. The book in plain text format can be downloaded from [Project Gutenberg website ](https://www.gutenberg.org/ebooks/2600).

Download and load data.

In [3]:
!wget "https://www.gutenberg.org/files/2600/2600-0.txt"

--2021-11-14 18:04:16--  https://www.gutenberg.org/files/2600/2600-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3359408 (3.2M) [text/plain]
Saving to: ‘2600-0.txt.4’

2600-0.txt.4        100%[===================>]   3.20M  2.14MB/s    in 1.5s    

2021-11-14 18:04:21 (2.14 MB/s) - ‘2600-0.txt.4’ saved [3359408/3359408]



In [4]:
# open text file and read in data as `text`
with open('2600-0.txt', 'r') as f:
    text = f.read()

Let's view the first 100 symbols:

In [5]:
text[:100]

'\ufeffThe Project Gutenberg eBook of War and Peace, by Leo Tolstoy\n\nThis eBook is for the use of anyone a'

The book starts at `7477` (the range was corrected due to a different version of the book):

In [6]:
text[7477:7700]

'CHAPTER I\n\n“Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don’t tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by that\n'

## Step 1. Tokenization (15 points)

Now let's create dictionaries for converting characters to integers and vice versa. This makes it easier to use symbols as input on the neural network.

In [7]:
# Create two dictionaries:
# int2char -- maps integers to characters
# char2int -- maps characters to unique integers

## YOUR CODE HERE
int2char = dict(enumerate(set(text)))   # maps integers to characters
char2int = {value: key for key, value in int2char.items()}   # maps characters to unique integers

# encode the text
encoded = np.array([char2int[ch] for ch in text])

Let's see how the encoding of characters into integers looks like

In [8]:
encoded[:100]

array([ 74,  59,  65,  42,  31, 100,  12,  38,   3,  42,  79,  32,  31,
         4,  56,  32,  42,  50,  27,  42,  12,   8,  31,  42, 106,  38,
        38,  73,  31,  38,   1,  31, 110, 109,  12,  31, 109,  50,  85,
        31, 100,  42, 109,  79,  42,  24,  31,  27,  58,  31,  69,  42,
        38,  31,  59,  38,  18,  98,  32,  38,  58, 104, 104,  59,  65,
         5,  98,  31,  42, 106,  38,  38,  73,  31,   5,  98,  31,   1,
        38,  12,  31,  32,  65,  42,  31,  56,  98,  42,  31,  38,   1,
        31, 109,  50,  58,  38,  50,  42,  31, 109])

## Step 2. One-hot encoding (15 points)

LSTM expects one-hot encoded input, which means that each character is converted to an integer (via our created dictionary) and then converted to a vector, where a value 1 will be only on a corresponding position and the rest of the vector will be filled with zeros.

Implement a function that does this kind of coding.


In [9]:
def one_hot_encode(arr, n_labels):
    
    # arr - array of integers
    # n_labels - number of labels (the size of a one-hot-encoded vector)

    ## YOUR CODE HERE
    one_hot = np.eye(n_labels)[arr]
    
    return one_hot

In [10]:
# check that the function works correctly
test_indx = np.array([[7, 2, 5]])
one_hot = one_hot_encode(test_indx, 8)

print(one_hot)

[[[0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]]]


The output must be

```
[[[0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]]]
```

## Step 3. Mini-batches (20 points)

To train the neural network, we will organize mini-batches as follows: divide the entire input sequence 'arr' by the desired number of subsequences (parameter `batch_size`), and send a sequence of length `seq_length` to the input of the network.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>


### How to create mini-batches


1.   Find the total number of batches `n_batches` that fit the text.
2.   Discard all unnecessary text that does not fit into full batches.
3.   Split text into `n_batches` batches'.
4.   Get `x's` and `y`'s for a batch. Therefore, `y` is a version of `x` shifted by 1.



In [11]:
def get_batches(arr, batch_size, seq_length):
    # Create a generator that returns batches of size batch_size x seq_length
    
    batch_size_total = batch_size * seq_length
    ## Get the number of batches we can make
    n_batches = len(arr) // batch_size_total ## YOUR CODE HERE
    
    ## Keep only enough characters to make full batches
    arr = arr[: n_batches * batch_size_total] ## YOUR CODE HERE
    
    ## Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1)) ## YOUR CODE HERE
    
    ## Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:(n+seq_length)]  ## YOUR CODE HERE
        
        # The target is a version of x shifted by one (do not forget border conditions)
        ## YOUR CODE HERE
        y = np.zeros_like(x)
        y[:, :-1] = x[:, 1:]
        y[:, -1] = arr[:, (n+seq_length)] if (n+seq_length) < arr.shape[1] else arr[:, 0]
        
        yield x, y

Let's check how our function works.

In [12]:
batches = get_batches(encoded, 4, 30)
x, y = next(batches)

In [13]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 74  59  65  42  31 100  12  38   3  42]
 [ 12 103  98 104  65  42 109  18  32  65]
 [ 27  42  12   5  50   8  31  65   5  98]
 [109  50  85  18  42  31  16   5  32  65]]

y
 [[ 59  65  42  31 100  12  38   3  42  79]
 [103  98 104  65  42 109  18  32  65  31]
 [ 42  12   5  50   8  31  65   5  98  31]
 [ 50  85  18  42  31  16   5  32  65  31]]


If you've implemented get_batches correctly, the result should look something like this (exact numbers may vary):
```
x
 [[ 48  94  12 110  32  96 107  34  18 106]
 [ 96  78   1  69  32  34  18  34  27   3]
 [  3  96   0  18  96  86  18 100  34  96]
 [ 32  96 110  42 101  96   0  18  19  96]]

y
 [[ 94  12 110  32  96 107  34  18 106  32]
 [ 78   1  69  32  34  18  34  27   3  94]
 [ 96   0  18  96  86  18 100  34  96   1]
 [ 96 110  42 101  96   0  18  19  96   3]]
 ```
Make sure the data is shifted one step for `y`.

## Step 4. Defining the network (40 points)

### Model Architecture

The proposed architecture is as follows:

* Define an [LSTM layer](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) with `dropout=drop_prob` and `batch_first=True` (since we use batches)
* Define a [Dropout layer](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html#torch.nn.Dropout) with `drop_prob`.
* Define a [Linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) with `in_features=n_hidden` and `out_features` equals to number of characters.

In [14]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU


In [15]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # Define the LSTM layer
        ## YOUR CODE HERE
        self.lstm = nn.LSTM(input_size=len(self.chars), hidden_size=self.n_hidden, 
                            num_layers=self.n_layers, batch_first=True, dropout=self.drop_prob)

        # Define a dropout layer
        ## YOUR CODE HERE
        self.dropout = nn.Dropout(self.drop_prob)

        # Define the final, fully-connected output layer
        ## YOUR CODE HERE
        self.linear = nn.Linear(in_features=self.n_hidden, out_features=len(self.chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        # Get the outputs and the new hidden state from the lstm
        ## YOUR CODE HERE
        out, hidden = self.lstm(x.float(), hidden)

        # Pass through a dropout layer
        ## YOUR CODE HERE
        out = self.dropout(out)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        ## YOUR CODE HERE
        out = out.reshape(out.size()[0] * out.size()[1], self.n_hidden)

        # Put x through the fully-connected layer
        ## YOUR CODE HERE
        out = self.linear(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Step 5. Train the model (10 points)

We use Cross Entropy as a loss function, Adam as optimizer, and [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to get rid of the gradient explosion.

In [16]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net(inputs, h)
            
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

### Model_1: Two LSTM-layers
n_hidden = 256  
n_layers = 2  
drop_prob=0.5  
batch_size = 64  
seq_length = 32

Initialize the model and set hyperparameters.

In [25]:
# Set your model hyperparameters

## YOUR CODE HERE
n_hidden = 256
n_layers = 2

net = CharRNN(tuple(set(text)), n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(112, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=256, out_features=112, bias=True)
)


Set training hyperparameters.

In [26]:
## YOUR CODE HERE
batch_size = 64
seq_length = 32
# start small if you are just testing initial behavior
n_epochs = 20

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.3538... Val Loss: 3.1813
Epoch: 1/20... Step: 20... Loss: 3.2587... Val Loss: 3.1136
Epoch: 1/20... Step: 30... Loss: 3.1573... Val Loss: 3.1075
Epoch: 1/20... Step: 40... Loss: 3.2028... Val Loss: 3.1137
Epoch: 1/20... Step: 50... Loss: 3.1946... Val Loss: 3.1056
Epoch: 1/20... Step: 60... Loss: 3.1751... Val Loss: 3.1033
Epoch: 1/20... Step: 70... Loss: 3.2426... Val Loss: 3.1071
Epoch: 1/20... Step: 80... Loss: 3.2324... Val Loss: 3.1032
Epoch: 1/20... Step: 90... Loss: 3.1475... Val Loss: 3.1008
Epoch: 1/20... Step: 100... Loss: 3.1589... Val Loss: 3.1007
Epoch: 1/20... Step: 110... Loss: 3.2132... Val Loss: 3.1000
Epoch: 1/20... Step: 120... Loss: 3.1791... Val Loss: 3.0991
Epoch: 1/20... Step: 130... Loss: 3.2001... Val Loss: 3.0992
Epoch: 1/20... Step: 140... Loss: 3.1391... Val Loss: 3.0955
Epoch: 1/20... Step: 150... Loss: 3.1542... Val Loss: 3.0933
Epoch: 1/20... Step: 160... Loss: 3.1142... Val Loss: 3.0891
Epoch: 1/20... Step: 170... Loss:

## Step 6. Making Predictions

Now that the model is trained, we want to predict the next character for the sequence of input symbols. We pass a character as input, and the network predicts the next character. Then we take that character, pass it as input, and get another predicted character, and so on.

### Details of a `predict`  function

Our RNN's output comes from a fully connected layer and outputs the **distribution of the next character scores**. To actually get the next character, we use the softmax function, which gives us a **probability** distribution that we can then choose to predict the next character.

**Top K sampling**

Our predictions are based on a categorical distribution of probabilities for all possible characters. We can make the sampling process smarter by looking at only some of the most likely $K$ characters. This will prevent the network from giving us completely absurd characters, and will also allow some noise and randomness to be introduced into the selected text. This technique is called [top K](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk) sampling.

In [28]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

To generate the text, you need to feed the initial characters, let's call them `prime`.

In [29]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

#### Making Predictions

In [30]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna Pávlovna was silent to the prince.

“Well, who is
the most marches... I
want?” the
patted smull the hair.

“What did you have been
soldiers, I should be bring to an officer,” said Denísov, trying to return
on the course there were a power to such bring
her and all his staff of his sang, and the commander
of the second pates talking it.

“Ah,
what and he does not though this time this is the princess! When it would be to be time to say it to still
the pleasure that the table.”

“You know, I do not, there!” cried Denísov’s hinds. “To the service to him to a sendence, it is a continuous cries of the commander’s
friend, the countess went on to see a face. It was a mild time all the time in the seminor and a few
man is as an offer single, and said to the capture, who alone assuged her.

“What a man the commander and was the significance!”

“I am a man answered in the party of the personal charming and a perplexed
music way. We will distate with me,” he
said, and several simple.

As if 

-------------

### Model_2: Two LSTM-layers
n_hidden = 1024  
n_layers = 2  
drop_prob=0.5   
batch_size = 128  
seq_length = 128

In [47]:
# Set your model hyperparameters

## YOUR CODE HERE
n_hidden = 1024
n_layers = 2
drop_prob=0.5

net = CharRNN(tuple(set(text)), n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(112, 1024, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=1024, out_features=112, bias=True)
)


In [48]:
## YOUR CODE HERE
batch_size = 128
seq_length = 128
# start small if you are just testing initial behavior
n_epochs = 20

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.1969... Val Loss: 3.1200
Epoch: 1/20... Step: 20... Loss: 3.1645... Val Loss: 3.1006
Epoch: 1/20... Step: 30... Loss: 3.1255... Val Loss: 3.0878
Epoch: 1/20... Step: 40... Loss: 3.1580... Val Loss: 3.0873
Epoch: 1/20... Step: 50... Loss: 3.1372... Val Loss: 3.0815
Epoch: 1/20... Step: 60... Loss: 3.1241... Val Loss: 3.0679
Epoch: 1/20... Step: 70... Loss: 3.0970... Val Loss: 3.0537
Epoch: 1/20... Step: 80... Loss: 3.0362... Val Loss: 2.9766
Epoch: 1/20... Step: 90... Loss: 2.9786... Val Loss: 2.9444
Epoch: 1/20... Step: 100... Loss: 2.9401... Val Loss: 2.8677
Epoch: 1/20... Step: 110... Loss: 2.8098... Val Loss: 2.7358
Epoch: 1/20... Step: 120... Loss: 2.6859... Val Loss: 2.6211
Epoch: 1/20... Step: 130... Loss: 2.5672... Val Loss: 2.5168
Epoch: 1/20... Step: 140... Loss: 2.5003... Val Loss: 2.4472
Epoch: 1/20... Step: 150... Loss: 2.4102... Val Loss: 3.1818
Epoch: 1/20... Step: 160... Loss: 2.4001... Val Loss: 2.3528
Epoch: 1/20... Step: 170... Loss:

#### Making Predictions

In [49]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna
Pávlovna’s handsome face, she was not at the same time, as a calam
to back his hands and a shappy mood in his fach, without like him, and so often this
and we are wanting for a moment in the marniage, but wished to say, and the
capital of having seeing Bezúkhov to the story was standing on his sister’s
hand.

“Well, all will be done in your cousin.
I will take the more than our little collasting,””
rated Pierre also. “You know the Emperor alone should be settled
in the same, only an officer with a men and such a special mising whell the
countess wants to speak of all the conversation with the wind, and had
to bran at all a military country, but they took them and who was
not the prait and has no officer and we are with my family and that is the
commin in the conversation was impossible to ask them all to be the whole monarch to
the princess. Had he not to be sent for his friend’s for the former things
to attack him, when he had to break it in at timide and directed the poor
prince

----

### Model_3: Three LSTM-layers
n_hidden = 1024  
n_layers = 3  
drop_prob=0.4  
batch_size = 254  
seq_length = 128

In [37]:
# Set your model hyperparameters

## YOUR CODE HERE
n_hidden = 1024
n_layers = 3
drop_prob=0.4

net = CharRNN(tuple(set(text)), n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(112, 1024, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=1024, out_features=112, bias=True)
)


In [38]:
## YOUR CODE HERE
batch_size = 254
seq_length = 128
# start small if you are just testing initial behavior
n_epochs = 20

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.1959... Val Loss: 3.1057
Epoch: 1/20... Step: 20... Loss: 3.1809... Val Loss: 3.0966
Epoch: 1/20... Step: 30... Loss: 3.1537... Val Loss: 3.0911
Epoch: 1/20... Step: 40... Loss: 3.1580... Val Loss: 3.0846
Epoch: 1/20... Step: 50... Loss: 3.1590... Val Loss: 3.0859
Epoch: 1/20... Step: 60... Loss: 3.1350... Val Loss: 3.0833
Epoch: 1/20... Step: 70... Loss: 3.1156... Val Loss: 3.0836
Epoch: 1/20... Step: 80... Loss: 3.1320... Val Loss: 3.0826
Epoch: 2/20... Step: 90... Loss: 3.1468... Val Loss: 3.0829
Epoch: 2/20... Step: 100... Loss: 3.1459... Val Loss: 3.0823
Epoch: 2/20... Step: 110... Loss: 3.1247... Val Loss: 3.0604
Epoch: 2/20... Step: 120... Loss: 3.0901... Val Loss: 3.0277
Epoch: 2/20... Step: 130... Loss: 2.9768... Val Loss: 2.9134
Epoch: 2/20... Step: 140... Loss: 2.9088... Val Loss: 2.8268
Epoch: 2/20... Step: 150... Loss: 2.8409... Val Loss: 2.7761
Epoch: 2/20... Step: 160... Loss: 2.8133... Val Loss: 2.7301
Epoch: 2/20... Step: 170... Loss:

#### Making Predictions

In [39]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna Pávlovna was already contented.

“You are gloamy and save your ball, in the same time if you have not?” his face
added: “Well.”





CHAPTER XIV

On the first time he sat down on a small position of her state, and a cheerful proper firilla and at the count had at once lasted his country with a commander who had been absent. The state of the count, who were carry in with her as if to talk in the fact as
it
was sitting on the firm. The cannon, were trying to repore to himself: “I have seen
those more steps off! How
did you want to,” said Prince Andrew, said something: “I shall not hear of this son. We shall not see you. If they are to serve.” She
recalled the soldiers who,
were at last as a conversation and tormented him. He was a single ance of water which had been straightened, and the face, but he did not rose and wait in the
door.





CHAPTER XX

In the stafes, and the people at the campaign were to be an excited
cap and the pleasure, which had so properly, but she was, as a ma

---

*Prepared by Sergey Afanasiev*